在 3.3 节（“线性回归的简洁实现”）一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [2]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

x = tf.random.uniform((2,20))
y = net(x)
y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.03759777,  0.0363857 , -0.07885749,  0.37808776,  0.16086985,
        -0.11086445, -0.10533479, -0.03453969, -0.03110847, -0.09440935],
       [ 0.0762679 ,  0.28920075, -0.23140612,  0.23718584,  0.06731918,
        -0.04753082, -0.08076701, -0.23043117, -0.00155275,  0.14943686]],
      dtype=float32)>

## 4.2.1 access model parameters

对于使用`Sequential`类构造的神经网络，我们可以通过weights属性来访问网络任一层的权重。回忆一下上一节中提到的`Sequential`类与`tf.keras.Model`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`tf.keras.Model`类的`weights`属性来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。**索引0表示隐藏层为`Sequential`实例最先添加的层**。

In [3]:
net.weights[0], type(net.weights[0])

(<tf.Variable 'dense/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[-0.02309236,  0.00859623, -0.00887281, ...,  0.07166921,
         -0.04473343,  0.02782936],
        [ 0.03154354,  0.12513971,  0.09200527, ..., -0.04814032,
         -0.11206327, -0.0521519 ],
        [ 0.09607175,  0.13451692, -0.04800403, ...,  0.10217629,
          0.08820492, -0.00875352],
        ...,
        [ 0.1202696 ,  0.07559642, -0.01066279, ..., -0.12693098,
          0.02862199,  0.08710681],
        [-0.09308457,  0.12082946,  0.10119343, ..., -0.03365725,
         -0.13185829, -0.03298087],
        [-0.08339414,  0.09513682, -0.12097948, ...,  0.13996911,
         -0.06528038, -0.10853954]], dtype=float32)>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

## 4.2.2 initialize params

我们在[“数值稳定性和模型初始化”]一节中描述了模型的默认初始化方法：**权重参数**元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [8]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
        units = 10,
        activation=None,
        kernel_initializer = tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()
        )
        self.d2 = tf.keras.layers.Dense(
        units=1,
        activation=None,
        kernel_initializer=tf.ones_initializer(),
        bias_initializer=tf.ones_initializer()
        )
        
    def call(self, input):
        output = self.d1(input)
        output = self.d2(output)
        return output

In [9]:
net = Linear()
net(x)
net.get_weights()

[array([[-1.08811492e-03, -4.38457914e-03, -8.26722104e-03,
         -6.10478688e-03, -3.90981324e-03,  3.16697778e-03,
         -4.52795485e-03, -2.45414185e-03, -2.40403172e-02,
          9.11507849e-03],
        [ 4.71600564e-03, -5.77836623e-03, -1.68437546e-04,
         -1.43275911e-03,  1.03372587e-02, -3.22884734e-04,
         -6.07993035e-03,  1.17144249e-02, -2.34356453e-03,
         -2.53304606e-03],
        [ 6.66316657e-04, -3.55211785e-03,  2.49082362e-03,
          1.96073297e-03,  4.54705814e-03, -7.43058138e-03,
         -3.45519744e-03,  1.44134341e-02,  1.27158137e-02,
          1.93931488e-03],
        [ 1.36847831e-02, -2.13804748e-02,  1.18884742e-02,
         -5.07963309e-03,  3.60178854e-03,  5.53009193e-03,
          7.37430714e-03, -1.62514914e-02, -6.11071382e-03,
         -4.36094869e-03],
        [ 8.16185668e-04,  5.18461037e-03, -9.74414963e-03,
         -7.24233501e-03,  7.20785977e-03, -1.95316896e-02,
         -4.96150553e-03,  1.05671771e-02,  1.308974

## 4.2.3 define initializer

可以使用`tf.keras.initializers`类中的方法实现自定义初始化。

In [10]:
def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

Y  = model(x)
model.weights[0]

<tf.Variable 'dense_5/kernel:0' shape=(20, 64) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_deep-learning-computation/parameters.html)

